In [36]:
import pandas as pd

In [37]:
census_df = pd.read_csv("./acs2017_census_tract_data.csv", 
                        usecols=['County', 'State', 'TotalPop', 
                                 'Poverty', 'IncomePerCap', 'TractId'])
covid_df = pd.read_csv("./COVID_county_data.csv", usecols=['date','cases','deaths'])

census_renamer = {
    "County": "Name",
    "TotalPop": "Population",
    "IncomePerCap": "PerCapitaIncome",
    "TractID": "ID",
}

covid_renamer = {
    "date": "Month",
    "cases": "Cases",
    "deaths": "Deaths",
}

In [38]:
census_df = census_df.rename(columns=census_renamer)
covid_df = covid_df.rename(columns=covid_renamer)

In [66]:
print(census_df.columns)
print(covid_df.columns)

Index(['TractId', 'State', 'Name', 'Population', 'PerCapitaIncome', 'Poverty'], dtype='object')
Index(['Month', 'Cases', 'Deaths'], dtype='object')


In [76]:
census_list = census_df[["Name", "State"]].copy()
state_county_list = census_list.drop_duplicates()
state_county_list.insert(2, "Population", 0)
state_county_list["Population"] = census_list.apply(lambda census_list: ), axis=1)


ValueError: 3

In [69]:
state_county_list.head(2)

,State,Name
0,Alabama,Autauga County
12,Alabama,Baldwin County


In [26]:
covid_df

,Month,Cases,Deaths
0,2020-01-21,1,0.0
1,2020-01-22,1,0.0
2,2020-01-23,1,0.0
3,2020-01-24,1,0.0
4,2020-01-24,1,0.0
...,...,...,...
1050253,2021-02-20,3645,34.0
1050254,2021-02-20,3318,9.0
1050255,2021-02-20,2024,12.0
1050256,2021-02-20,876,26.0
